### Предложения по проекту

#### Замечания:

    1. Так как в данных присутствует временная составляющая, то почему бы не рассматривать его их как временные и соответственно не пользоваться сответствующими методами
    
    2. Если смотреть с этой стороны то можно сделать предположение, что данные, которые ближе к текущему моменту несут больше информации, чем те, что были зафиксированы раньше, что , например, справедливо для случая, который он нам приводил как проблемный.
    
    "Человек жил в одном городе, но потом перехал или просто поехал в другой и соответственно наше соответствующее среднее значение застревает между городам и не даёт какой-либо полезной информации"
    
    3. В методе который они обговаривали (формирование соответствующей точки для каждого клиента через среднюю или медиану временной фактор не учитывается и более старые данные по ценности равны более свежим, что может сильно смещать картину в сторону рестроспективы)

#### Предлагаемая эвристика

    Как было написано выше, необходим метод, в котором более свежие данные будут цениться выше, чем старые, то есть вес, с которым новая точка входит должен отличаться от остальных, во многих задаччах связанных с временем используется сглаживание (экспоненциальное сглаживание или экспоненциальная скользящая средняя), с этой позиции я и предлагаю взглянуть на проблему
    
    Экспоненциальное сглаживание:
    
$$ S_t = \alpha*x_t+(1-\alpha)*S_{t-1} $$

    где:

   1. $x_t$ - наблюдаемое значение координаты в момент времени t
   2. $S_t$ - соответствующее значение точки для клиента в момент времени t (далее глобальная координата)
   3. $\alpha$ из (0,1]
    
    Однако, обычный временной ряд характеризуется тем, что наблюдения производятся через равные промежутки времени и соответственно с помощью различных общих представлений $\alpha$ ,подбирается какой-нибудь константой, например, 0.15, что не выполняется для наших данных, то есть было бы нелогичным взять а = const так как очевидно, что например в случае, когда у нас , скажем, два наблюдения и в одном случае между ними прошёл час, а в другом 20 дней, то и обновление глобальной координаты по средством а должно быть различным.
    Данное замечание наводит на мысль, почему бы не формировать а, основываясь на длине интервала времени, прошедшем до текущего момента после предыдущего платежа, то есть
    
   $$\alpha = time_t - time_{t-1}$$
   
    Тут однако возникает проблемы:
    
   1. В чём проводить измерения: минуты, секунды, часы итд.
    
   2. Время возрастает линейно, что может сулить проблемами в будущем
    
   3. $\alpha$ не принадлежит интервалу (0,1]
    
    Если с первой проблемой можно справится, если согласиться конвенционально использовать минуты, так как количество платежей, между которыми проходит меньше минуты крайне мало и к тому же в секундах значения разности в 60 раз больше, что также может доставить не мало проблем.
    Что касается второй проблемы, то я бы предложил использовать не просто разность а логарифм разности, что  будет сглаживать черезмерный рост и не позволит альфа слишком быстро возрастать( об основании логарифма дальше)
    
$$\alpha = \log_\kappa(\kappa+(time_t-time_{t-1}))$$
    
   $\kappa$ нужно добавлять в логарифм, чтобы если прошло меньше минут, чем $\kappa$, то мы не получали бы отрицательные значения или $\infty$, если прошло меньше минуты
     
     Что касается третьей проблемы, то я бы предложил использовать сигмоиду, чтобы заключить альфа в нужный интервал
     
$$\alpha = \theta(\log_\kappa(\kappa+(time_t-time_{t-1})))$$

    Если тут вспомнить, как выглядит график сигмоиды, то вспоминается, что в нуле она равна 0.5, а принимая во внимание, что мы определяем её на логарфме, который в нашем случае принимает, только занчения >= 0, то получается, что мы ограничиваем снизу альфа 1/2,что не выглядит как хороший вариант, поэтому модифицируем формулу следующим образом:
    
    
$$\alpha = \theta(\log_\kappa(\kappa+(time_t-time_{t-1}))-\beta)$$

    Где полагается взять бета такой, что елси напримерпрошло меньше минуты, то мы получим
   
$$
\alpha = \theta(x)\\
x=1-\beta
$$

    Так как за минуту человек вряд ли успел переместиться на расстояние большее, чем метров сто, то вряд ли следует как-то значительно обновлять положение глобальной координаты, тогда предположим, что альфа принимает одно из следующих значений
    
$$
\alpha = 0.01  \rightarrow \beta = 5.6\\
\alpha = 0.001 \rightarrow \beta = 7.9
$$

    Выбрать можно на свой вкус
    
    Что касается основания логарифма, то я бы предложил выбрать основываясь на следующем.
    Допустим, что у нас между двумя последовательными наблюдениями прошел некий конвенциональный переиод, скажем 21 день, это значение не строгое, и тогда основываясь на нем, допустим, что раз прошёл такой "большой" промежуток времени, мы говорим, что предыдущее значение глобальной координаты сильно устарело и более вряд ли несёт в себе какую-то действительно полезную информацию и мы хотим получить для нового наблюдения значение альфа из набора скажем {0.85, 0.9, 0.95} это значение обознчим через гамма (можно и меньше, чтобы меньше забывать предыдущую глобальную координату) Тогда получим:
    
    В 21 дне 30240 минут (функция для вычисления примерного значения приведена в ячейке ниже)
    
$$
\beta = 5.6, \gamma = 0.85 \rightarrow \kappa = 4.08\\
\beta = 5.6, \gamma = 0.90 \rightarrow \kappa = 3.75\\
\beta = 5.6, \gamma = 0.95 \rightarrow \kappa = 3.34\\
\\
\beta = 7.9, \gamma = 0.85 \rightarrow \kappa = 2.92\\
\beta = 7.9, \gamma = 0.90 \rightarrow \kappa = 2.77\\
\beta = 7.9, \gamma = 0.95 \rightarrow \kappa = 2.59\\
$$
    
    Это что касается представления пользователя в виде точки на карте в глобальном масштабе, то есть для кластеризации по всей стране, не на локальном уровне(в пределах одного города)
    
    Ещё как задумка, взять какой-то фиксированный промежуток времени, скажем две недели(можно меньше, можно больше), рассчитать относительно какой-либо точки в этом промежутке, начала отсчёта(первое наблюдение или просто точка начала отсчёта)) расстояние временное для каждого платежа через тот же самый логарифм, засунуть всё это дело в софтмакс, потом с полученными весами сложить и получить кглобальную координату, я сосбо не думал над этим вариантом)
    
    Одним из главных преимуществ всего подхода видится то, что для обновления глобальной координаты нет необходимости обращаться к предыдущим положениям.
    

### Замеченная проблема

    Как видно из экспериментов первоначальное предположение логарифм в начале возрастает линйено, а потом начинает относительно медленно возрастать, что приводит к следующим последствиям:
    
    1. В начале альфа возрастает достаточно быстро и за день может добежать до значений в 0.5 - 0.7
    2. Далее же рост становится медленным и за двадцать дней он доходит премерно до 0.9 в зависимости от начальных параметров
    
    Нам же нужно другое поведение, то есть за скажем 1 неделю изменение не должно превышать скажем 0.5, а дальше оно должно быстро возрастать, такое поведение может в принципе обеспечивать степенная функция. Однако начнем с другого, определимся, что нам глобально надо, нам нужна функция f(x) такая что:

$$\theta(f(x))==0.1, where x == 1440 \\
   \theta(f(x))==0.5, where x == 10080$$
   
   1440 - минут в дне и 10080 минут в неделе, дни и пороги взяты с пустого места, чисто для начала
   
   Если искать подходящую функцию в классе линейных
$$f(x)== a*x+b$$
    Тогда:
$$
a==0.000254308\\
b==-2.56343
$$

$$
928: minutes\\
alpha: 0.5025233109579226\\
alpha_2: 0.08887637948669712\\
$$

$$
30240: minutes\\
alpha: 0.8444490702360951\\
alpha_2: 0.994099756388402\\
$$

    Как видно, в случае логарифма альфа быстро возрастает до 0.5, однако дальнейший рост довольно затруднителен, в то время как у линейной фнкции все иначе, её неподвижность в начале при перемещени в другой удалённый город за день не даст ей быстро к нему сойтись, что как отмечалось ранее было изначальной проблемой, поэтому такой подход к решение не логичен. Однако попробуем увеличить порог для альфа за один прошедший день до 0.3
$$    
928 minutes
alpha == 0.5025233109579226
alpha_2 == 0.2895628903402183
lat:  43.581657310647685 & lon:  29.333436362574705
lat_2:  48.484146027377385 & lon_2:  32.67727361585287
$$

$$
1440 minutes
\alpha == 0.5550794814566932
\alpha_2 == 0.3000000412413144
lat:  34.49500650266635 & lon:  23.057936301888276
lat_2:  40.08513132913285 & lon_2:  26.89762268397703
$$

    Как видно в предложенном далее примере за три дня линейная модель не успевает сойтись и оставляет точку где-то на половине пути.
    
    Также увеличив дальше до 0.4 стала абсолютно очевидной ещё одна проблема, из-за очень маленького коэффициента при минутном расстоянии разница в обновлениях за два часа и за день крайне мала и оба обновления примерно равны 0.3, что явно явлется не тем результатом, которого хотелось добиться.
    
    Тут явно проявляется ограниченность линейности, попробуем несколько расширить свободу действий и заменить на квадратичную со следующими попрогами:
    
    1. неделя - 0.65
    2. день - 0.4
    3. час - 0.05
    
$$ 
f(x) = a*x^2+b*x+c
$$

    Тогда получаем, что:
$$
    a = -1.71782e-7\\
    b = 0.00209751\\
    c = -3.06967
$$

$$
928: minutes
alpha: 0.5025233109579226
alpha_2: 0.24539437418878857
lat:  43.581657310647685 & lon:  29.333436362574705
lat_2:  49.74549727191266 & lon_2:  33.36553564920472
$$

$$
1440: minutes
alpha: 0.5550794814566932
alpha_2: 0.48759959156643157
lat:  34.49500650266635 & lon:  23.057936301888276
lat_2:  39.423145499325585 & lon_2:  26.364944415977952
$$

    Какого-то качественного улучшения замечено не было, если есть идеи на счёт модификации или экспериментами с попрогами, то welcome
    
    
    На фоне всех этих размышлений возник вопрос, а собственно почему функция f зависит только от времени и никак от расстояния. Ведь если у нас прошло много времени, но расстояние нового платежа от глобальной координаты не сильно большое, то нет какой-то весомой причины сильно обновлять глобальную координату и наоборот, если времени прошло не так много, но расстояние большое, то было бы логично применить большое обновление.
    
    То есть:
$$
\alpha = \theta(f(d(time_t,time_{t-1}), d(coordinate_t, globalСoordinate_{t-1}))
$$

    Обратимся к старой формуле
    
$$\alpha = \theta(\log_\kappa(\kappa+(time_t-time_{t-1}))-\beta)$$

    Вопервых, необходимо определиться с d(coordinate_t, globalСoordinate_{t-1}), моё предложение заключается в 
$$Log_{10}(dist(coordinate_t, globalСoordinate_{t-1}))$$
    Где dist - расстояние в километрах. Другой вопрос, как ввести в формулу данное соотношение, очевидно, что:
        
        1. Оно должно быть внутри сигмоиды или если снуружи, то должно входить в отдельную сигмоиду и скорее всего они должны скалыдваться, только с какими коэффициентами не очень ясно
        2. Если заносить внутрь, то перемножение вряд ли будет являться хорощей опцией, потому что тогда аргумент сигмоиды может легко стать очень большим, что ни в коем случае не играет нам на руку, поэтому опять надо складывать (на мой взгляд), тут снова встаёт вопрос с какими коэффициентами и вообще по хорошему, надо пересчитытвать в таком случае основание логарифма и параметр бета из старой формулы
        
        Предлагаемые варианты:
        
1 Вариант: 
$$\alpha = \theta(\zeta*\log_\kappa(\kappa+(time_t-time_{t-1}))+\eta*\log_{10}(||Coord_t-gCoord_{t-1}||+1)-\beta)$$

2 Вариант:        
$$\alpha = \zeta*\theta(\log_\kappa(\kappa+(time_t-time_{t-1}))-\beta_1))+\eta*\theta(\log_{10}(||Coord_t-gCoord_{t-1}||+1)-\beta_2)$$ 

(В обоих случаях десятичный логарифм можно заменить, на другое основание). К логарифму прибаляется единица, чтобы избежать отрицательных значений, и чтобы не сильно влиять на получаемое значение

In [1]:
### Функция dist для измерения расстояния между двумя координатами

import math 
def step_osn(days, porog):
    #1-s^(days*24*60-base)==porog
    #1-s^(24*60-base)==porog2
    if(porog==0.5):
        print("Error, it must not be equal to 0.5")
        return
    tmp = porog**(-1)-1
    if(tmp>1):
        tmp_1 = math.log(tmp,math.e)
        return -math.log(tmp_1,days*24*60)
    else:
        tmp_1 = -math.log(tmp,math.e)
        return -math.log(tmp_1,days*24*60)
step_osn(7,0.49)

#!pip install haversine
from haversine import haversine, Unit

def dist_km(lat_1, lon_1, lat_2, lon_2):
    var_1 = (lat_1, lon_1)
    var_2 = (lat_2, lon_2)
    return haversine(var_1, var_2, Unit.KILOMETERS)

#### Дополнение:

        Хотя предложенный метод достаточно неплохо в теории должен справляться с долгосрочной релокацией человека и гораздо быстрее сходится к новой глобальной координате, как видно из примера ниже при выбранных параметрах всего за три дня он почти полностью переместился на новую точку, только вот три дня это много или мало, в день прилета точка выглядит очень странно и не даёт содержательной информации. Поэтому я думал о том, чтобы каждому человеку соответствовало не одна а две точки, каждая вычисляется описанным образом, но с разными параметрами, одна будет робастной и будет очень быстро следовать за человеком, другая же будет наоборот очень медленно сходится, в общем случае, когда человек не уезжает куда-то далеко они будут очень похожи и наиболее вероятно будут относится к одному кластеру, что не должно доставлять неприятностей.
        
         Однако в таком случае из-за медленной сходимости медленной координаты мы получим для неё промежуток времени, когда она движется к быстрой точке по местпам, которые ничего не дают, тут может быть несколько подходов, например, если расстояние между быстрой и медленной координатой больше, скажем, 100км, считаем достоверной быструю, в противном случае считаем верной медленную.
         
         Другой менее интуитывный подход состоит в том, чтобы считать медленную координату с задержкой в некий промежуток времени, допустим, что мы предполаем, что в среднем люди если и уезжают из дома, то на срок не превышающий две недели, такую релокацию будет считать незначимой и соответственно робастную(быструю) координату мы считаем как обычно, а для медленной обновления приходят с опозданием в две недели, то есть если мы сейчас получили платёж, то обновляем быструю координату, а медленную обновим координату этим платежом обновим через две недели, таким образом на время поездки, одна координата будет в месте постоянной дислокации, а другая в новом месте. 
         
         Тут правда происходит следующий момент, что медленная координата начнёт сходиться к временной координате через две недели, когда по нашему предположению человек уже вернется домой и эта координата станет бесполезной, поэтому тут есть такое предложение, как если человек за две или менее недели возвращается в домой, то платежи за этот переиод для медленной координаты скипаются, то есть для неё будто ничего и не произошло, возвращение описывается условием, допустим ||x_fast-x_slow||<=100км (это всё наброски пока). Если же за две недели человек не возвращается, то медленной координате присваивается значение координате, которой была равна быстра координата неделю назад, таким образом, мы оставляем некую ретроспективность медленной координаты и при этом минуем этам долгого схожденияк новой постоянной дислокации

    Для удобства:
        
1 Вариант: 
$$\alpha = \theta(\zeta*\log_\kappa(\kappa+(time_t-time_{t-1}))+\eta*\log_{10}(||Coord_t-gCoord_{t-1}||+1)-\beta)$$

2 Вариант:        
$$\alpha = \zeta*\theta(\log_\kappa(\kappa+(time_t-time_{t-1}))-\beta_1))+\eta*\theta(\log_{10}(||Coord_t-gCoord_{t-1}||+1)-\beta_2)$$ 

### Ниже приведён итерационный алгортим нахождения наилучших параметров 

In [26]:
# Полезные функции можно найти ниже, извини за такую сумбурность
# можешь поэксперементировать с параметрами в определении функции

import numpy as np
import math

def sigmoid(x):
    return 1/(1+math.exp(-x))

def sigmoid_dt(x,y, k = 3.3, beta = 5.6):
    #x,y - дата и время в виде строки, формат в функциях выше, разность всегда положительная
    # k - основание логарифма
    # b - beta
    k = k
    b = beta
    dist = minutes_between_high(y,x)
    #print(dist,'minutes')
    return 1.0/(1+math.exp(-(math.log(dist+k,k)-b)))

def sigmoid_dt_2(x,y):#для экспериментов с другими функциями (сейчас тут квадратичная)
    a = -1.71782e-10
    b = 0.00209751
    c = -3.06967
    dist = minutes_between_high(y,x)
    #print(dist,'minutes')
    return 1.0/(1+math.exp(-(a*dist*dist+b*dist+c)))

def sigmoid_dt_var_2(lat_1, lon_1, dtime_1, lat_2, lon_2, dtime_2, zeta = 0.3, eta = 0.7, beta_1 = 5.6, beta_2 = 4, k = 3.5):#вариант 1
    
    #Коэффициенты при сигмоидах
    zeta = zeta
    eta = eta
    
    
    #Свободный член
    beta_1 = beta_1
    beta_2 = beta_2
    
    #Основание логарифма
    k = k
    
    km = dist_km(lat_1, lon_1, lat_2, lon_2)
    dist = minutes_between_high(dtime_1, dtime_2)
    sigmoid_1 = sigmoid(math.log(dist+k,k)-beta_1)
    sigmoid_2 = sigmoid(math.log(km+1,10)-beta_2)
    #print(" ")
    #print("Вариант 2")
    #print(km, 'километров')
    #print(sigmoid_1, 'сигмоида времени')
    #print(sigmoid_2, 'сигмоида расстояния')
    
    
    return zeta*sigmoid_1+eta*sigmoid_2
    
def sigmoid_dt_var_1(lat_1, lon_1, dtime_1, lat_2, lon_2, dtime_2,zeta = 0.3, eta = 0.7, beta = 5, k = 3.5):# вариант 2
    
    #Коэффициенты при логарифмах
    zeta = zeta
    eta = eta
    # так как эти коэффициенты внутри сигмоиды, их сумма не обязательно должна быть равна 1
    
    #Свободный член
    beta = beta
    
    #Основание логарифма
    k = k
    
    km = dist_km(lat_1, lon_1, lat_2, lon_2)
    dist = minutes_between_high(dtime_1,dtime_2)
    log_t = math.log(dist+k,k)
    log_d = math.log(km+1,10)
    
    #print(" ")
    #print('Вариант 1')
    #print(km, 'километров')
    #print(log_t, 'логарифм времени')
    #print(log_d, 'логарифм расстояния')
    
    return sigmoid(zeta*log_t+eta*log_d-beta)



#____________________________________________________________________________________

mas = [[55.54,37.18,"13:43:22 2019-05-28"],
       [55.54,37.18,"13:43:22 2019-05-30"],
       [55.60,37.20,"13:43:22 2019-05-31"],
       [55.66,37.22,"13:43:22 2019-06-01"],#дома
       [55.20,37.78,"18:20:22 2019-06-01"],# в магазине
       [53.11,36.56,"05:23:12 2019-06-02"],# в аэропорте в своём
       [35.88,24.72,"20:51:31 2019-06-02"],# в аэропорте чужом
       [35.66,24.30,"08:12:36 2019-06-03"],# в отеле
       [35.66,24.30,"08:12:36 2019-06-04"]# в отеле
      ]

min_alpha = 1000000
min_alpha1 = 1000000
min_alpha2 = 1000000


min_alpha_k = -1
min_alpha_z = -1
min_alpha_e = -1
min_alpha_b = -1

min_alpha1_k = -1
min_alpha1_z = -1
min_alpha1_e = -1
min_alpha1_b = -1

min_alpha2_k = -1
min_alpha2_z = -1
min_alpha2_e = -1
min_alpha2_b1 = -1
min_alpha2_b2 = -1

for k in np.arange(1.5,10.0,0.5):
    for zeta in np.arange(0.05,1,0.05):
        eta = 1-zeta
        for beta_1 in np.arange(1,10,0.5):
            for beta_2 in np.arange(1,10,0.5):
                j=0
                a  = 0
                a1 = 0
                a2 = 0
                for i in mas:
                    lat = i[0]
                    lon = i[1]
                    t_d = i[2]
                    if(j == 0):
                        global_lat = lat
                        global_lon = lon

                        global_lat_var_1 = lat
                        global_lon_var_1 = lon

                        global_lat_var_2 = lat
                        global_lon_var_2 = lon

                        p_td = t_d
                        
                        plat = lat
                        plon = lon 
                        
                        j+=1

                    else:
                        
                        alpha = sigmoid_dt(p_td,t_d, k=k, beta = beta_1)
                        alpha_var_1 = sigmoid_dt_var_1(lat, lon, t_d, global_lat, global_lon, p_td, k = k, beta=beta_1, zeta = zeta, eta=eta)
                        alpha_var_2 = sigmoid_dt_var_2(lat, lon, t_d, global_lat, global_lon, p_td, k = k, beta_1=beta_1, beta_2=beta_2, zeta = zeta, eta=eta)
                        #alpha_2 = sigmoid_dt_2(p_td,t_d)

                        p_td=t_d

                        #print(" ")
                        #print('alpha:      ',alpha)
                        global_lat = alpha*lat+(1.0-alpha)*global_lat
                        global_lon = alpha*lon+(1.0-alpha)*global_lon

                        #print('alpha_var_1:',alpha_var_1)
                        global_lat_var_1 = alpha_var_1*lat+(1.0-alpha_var_1)*global_lat_var_1
                        global_lon_var_1 = alpha_var_1*lon+(1.0-alpha_var_1)*global_lon_var_1

                        #print('alpha_var_2:',alpha_var_2)
                        global_lat_var_2 = alpha_var_2*lat+(1.0-alpha_var_2)*global_lat_var_2
                        global_lon_var_2 = alpha_var_2*lon+(1.0-alpha_var_2)*global_lon_var_2

                        #print('alpha_2',alpha_2)
                        #global_lat_2 = alpha_2*lat+(1.0-alpha_2)*global_lat_2
                        #global_lon_2 = alpha_2*lon+(1.0-alpha_2)*global_lon_2
                        #print(" ")
                        
                        kilom = dist_km(lat,lon,plat,plon)
                        
                        if(kilom>2000):
                            lol = 0.8
                            xlat = plat + lol*(lat-plat)
                            xlon = plon + lol*(lon-plon)
                        elif(kilom<=2000 and kilom>1000):
                            lol = 0.7
                            xlat = plat + lol*(lat-plat)
                            xlon = plon + lol*(lon-plon)
                        elif(kilom<=1000 and kilom>500):
                            lol = 0.6
                            xlat = plat + lol*(lat-plat)
                            xlon = plon + lol*(lon-plon)
                        elif(kilom<=500 and kilom>250):
                            lol = 0.5
                            xlat = plat + lol*(lat-plat)
                            xlon = plon + lol*(lon-plon)
                        elif(kilom<=250 and kilom>100):
                            lol = 0.55
                            xlat = plat + lol*(lat-plat)
                            xlon = plon + lol*(lon-plon)
                        elif(kilom<=100 and kilom>50):
                            lol = 0.5
                            xlat = plat + lol*(lat-plat)
                            xlon = plon + lol*(lon-plon)
                        elif(kilom<=50 and kilom>10):
                            lol = 0.45
                            xlat = plat + lol*(lat-plat)
                            xlon = plon + lol*(lon-plon)
                        else:
                            lol = 0.4
                            xlat = plat + lol*(lat-plat)
                            xlon = plon + lol*(lon-plon)

                        a+= dist_km(xlat,xlon,global_lat,global_lon)
                        a1+= dist_km(xlat,xlon, global_lat_var_1, global_lon_var_1)
                        a2+= dist_km(xlat,xlon, global_lat_var_2, global_lon_var_2)
                        
                        plat = lat
                        plon = lon
                        
                        
                if(a<min_alpha):
                    min_alpha = a
                    min_alpha_k = k
                    min_alpha_b = beta_1

                if(a1<min_alpha1):
                    min_alpha1 = a1
                    min_alpha1_k = k
                    min_alpha1_z = zeta
                    min_alpha1_e = eta
                    min_alpha1_b = beta_1  

                if(a2<min_alpha2):
                    min_alpha2 = a2
                    min_alpha2_k = k
                    min_alpha2_z = zeta
                    min_alpha2_e = eta
                    min_alpha2_b1 = beta_1 
                    min_alpha2_b2 = beta_2

print('min alpha '+str(min_alpha)+' k = '+str(min_alpha_k)+' beta = '+str(min_alpha_b)) 
print('min alpha1 '+str(min_alpha1)+' k = '+str(min_alpha1_k)+' zeta = '+str(min_alpha1_z)+' eta = '+str(min_alpha1_e)+' beta = '+str(min_alpha1_b))
print('min alpha2 '+str(min_alpha2)+' k = '+str(min_alpha2_k)+' zeta = '+str(min_alpha2_z)+' eta = '+str(min_alpha2_e)+' beta_1 = '+str(min_alpha2_b1)+' beta_2 = '+str(min_alpha2_b2))
    #print('lat:       ',round(global_lat,6),      ' lon:       ', global_lon)
    #print('lat_var_1: ',round(global_lat_var_1,6),' lon_var_1: ', global_lon_var_1)
    #print('lat_var_2: ',round(global_lat_var_2,6),' lon_var_2: ', global_lon_var_2)
    #print('real_lat:  ',round(lat,6),   '      real_lon:  ', lon)
    #print(" ")
    #print('_'*100)
    #print(" ")

min alpha 173.14373326600136 k = 4.0 beta = 3.5
min alpha1 174.26959175884207 k = 3.0 zeta = 0.9000000000000001 eta = 0.09999999999999987 beta = 4.5
min alpha2 137.8060934216365 k = 9.0 zeta = 0.1 eta = 0.9 beta_1 = 4.0 beta_2 = 1.5


In [14]:
mas = [[55.54,37.18,"13:43:22 2019-05-28"],
       [55.54,37.18,"13:43:22 2019-05-30"],
       [55.60,37.20,"13:43:22 2019-05-31"],
       [55.66,37.22,"13:43:22 2019-06-01"],#дома
       [55.20,37.78,"18:20:22 2019-06-01"],# в магазине
       [53.11,36.56,"05:23:12 2019-06-02"],# в аэропорте в своём
       [35.88,24.72,"20:51:31 2019-06-02"],# в аэропорте чужом
       [35.66,24.30,"08:12:36 2019-06-03"],# в отеле
       [35.66,24.30,"08:12:36 2019-06-04"]# в отеле
      ]

def sigmoid_dt(x,y, k = 4.55, beta = 7.1, step_osn = 2.0):
    #x,y - дата и время в виде строки, формат в функциях выше, разность всегда положительная
    # k - основание логарифма
    # b - beta
    k = k
    b = beta
    dist = minutes_between_high(y,x)
    #print(dist,'minutes')
    return 1.0/(1+step_osn**(-(math.log(dist+k,k)-b)))

def sigmoid_dt_2(x,y):#для экспериментов с другими функциями (сейчас тут квадратичная)
    a = -1.71782e-10
    b = 0.00209751
    c = -3.06967
    dist = minutes_between_high(y,x)
    print(dist,'minutes')
    return 1.0/(1+math.exp(-(a*dist*dist+b*dist+c)))

def sigmoid_dt_var_2(lat_1, lon_1, dtime_1, lat_2, lon_2, dtime_2, zeta = 0.3,
                     eta = 0.7, beta_1 = 6.56, beta_2 = 2.58, k = 4.15, osn_step_1 = 3.9, osn_step_2=3.08):#вариант 2
    
    #Коэффициенты при сигмоидах
    zeta = zeta
    eta = eta
    
    
    #Свободный член
    beta_1 = beta_1
    beta_2 = beta_2
    
    #Основание логарифма
    k = k
    
    km = dist_km(lat_1, lon_1, lat_2, lon_2)
    dist = minutes_between_high(dtime_1, dtime_2)
    sigmoid_1 = 1/(1+osn_step_1**(-(math.log(dist+k,k)-beta_1)))
    sigmoid_2 = 1/(1+osn_step_1**(-(math.log(km+1,10)-beta_2)))
    print(" ")
    print("Вариант 2")
    print(km, 'километров')
    print(sigmoid_1, 'сигмоида времени')
    print(sigmoid_2, 'сигмоида расстояния')
    
    
    return zeta*sigmoid_1+eta*sigmoid_2
    
def sigmoid_dt_var_1(lat_1, lon_1, dtime_1, lat_2, lon_2, dtime_2,zeta = 0.4776, 
                     eta = 0.7551, beta = 5.5062, k = 3.1446, osn_step = 2.75):# вариант 2
    
    #Коэффициенты при логарифмах
    zeta = zeta
    eta = eta
    # так как эти коэффициенты внутри сигмоиды, их сумма не обязательно должна быть равна 1
    
    #Свободный член
    beta = beta
    
    #Основание логарифма
    k = k
    
    km = dist_km(lat_1, lon_1, lat_2, lon_2)
    dist = minutes_between_high(dtime_1,dtime_2)
    log_t = math.log(dist+k,k)
    log_d = math.log(km+1,10)
    
    print(" ")
    print('Вариант 1')
    print(km, 'километров')
    print(log_t, 'логарифм времени')
    print(log_d, 'логарифм расстояния')
    
    return 1/(1+osn_step**(-(zeta*log_t+eta*log_d-beta)))


j=0

for i in mas:
    lat = i[0]
    lon = i[1]
    t_d = i[2]
    if(j == 0):
        global_lat = lat
        global_lon = lon

        global_lat_var_1 = lat
        global_lon_var_1 = lon

        global_lat_var_2 = lat
        global_lon_var_2 = lon

        p_td = t_d
        j+=1

    else:

        alpha = sigmoid_dt(p_td,t_d)#, k=4.4, beta = 5.0, step_osn=6)#new_sigmoid(p_td,t_d)#
        alpha_var_1 = sigmoid_dt_var_1(lat, lon, t_d, global_lat, global_lon, p_td)#, k = min_alpha1_k, beta=min_alpha1_b, zeta = min_alpha1_z, eta=min_alpha1_e)
        alpha_var_2 = sigmoid_dt_var_2(lat, lon, t_d, global_lat, global_lon, p_td)#, k = min_alpha2_k, beta_1=min_alpha2_b1, beta_2=min_alpha2_b2, zeta = min_alpha2_z, eta=min_alpha2_e)
        #alpha_2 = sigmoid_dt_2(p_td,t_d)
        alpha = target_alpha_func(lat,lon,t_d,global_lat, global_lon, p_td)
        print("target alpha",alpha)

        p_td=t_d

        print(" ")
        print('alpha:      ',alpha)
        global_lat = alpha*lat+(1.0-alpha)*global_lat
        global_lon = alpha*lon+(1.0-alpha)*global_lon

        print('alpha_var_1:',alpha_var_1)
        global_lat_var_1 = alpha_var_1*lat+(1.0-alpha_var_1)*global_lat_var_1
        global_lon_var_1 = alpha_var_1*lon+(1.0-alpha_var_1)*global_lon_var_1

        print('alpha_var_2:',alpha_var_2)
        global_lat_var_2 = alpha_var_2*lat+(1.0-alpha_var_2)*global_lat_var_2
        global_lon_var_2 = alpha_var_2*lon+(1.0-alpha_var_2)*global_lon_var_2

            #print('alpha_2',alpha_2)
            #global_lat_2 = alpha_2*lat+(1.0-alpha_2)*global_lat_2
            #global_lon_2 = alpha_2*lon+(1.0-alpha_2)*global_lon_2
            #print(" ")

            #a+= dist_km(lat,lon,global_lat,global_lon)
            #a1+= dist_km(lat,lon, global_lat_var_1, global_lon_var_1)
            #a2+= dist_km(lat,lon, global_lat_var_2, global_lon_var_2)
        print(" ")
        print('lat:       ',round(global_lat,6),      ' lon:       ', global_lon)
        print('lat_var_1: ',round(global_lat_var_1,6),' lon_var_1: ', global_lon_var_1)
        print('lat_var_2: ',round(global_lat_var_2,6),' lon_var_2: ', global_lon_var_2)
        print('real_lat:  ',round(lat,6),   '      real_lon:  ', lon)
        print(" ")
        print('_'*100)
        print(" ")

 
Вариант 1
0.0 километров
6.953591134388344 логарифм времени
0.0 логарифм расстояния
 
Вариант 2
0.0 километров
0.2126788506396638 сигмоида времени
0.028991866850513293 сигмоида расстояния
target alpha 0.0825
 
alpha:       0.0825
alpha_var_1: 0.09881022431291753
alpha_var_2: 0.08409796198725844
 
lat:        55.54  lon:        37.18
lat_var_1:  55.54  lon_var_1:  37.18
lat_var_2:  55.54  lon_var_2:  37.18
real_lat:   55.54       real_lon:   37.18
 
____________________________________________________________________________________________________
 
 
Вариант 1
6.7891588424950395 километров
6.3495367334792565 логарифм времени
0.8914905603956493 логарифм расстояния
 
Вариант 2
6.7891588424950395 километров
0.12234992554182042 сигмоида времени
0.09128693081423603 сигмоида расстояния
target alpha 0.0932668783282257
 
alpha:       0.0932668783282257
alpha_var_1: 0.13926787031174523
alpha_var_2: 0.10060582923251135
 
lat:        55.545596  lon:        37.18186533756656
lat_var_1:  55.5483

In [16]:
import math 

def log_osn(days, beta, gamma):
    minutes = days*24*60
    tmp = beta-math.log(gamma**(-1)-1,math.e)
    print('Основание: '+str((minutes)**(1/tmp)))
    
log_osn(21, 7.9, 0.95)

Основание: 2.5892179592679554


### Низкоуровневая кластеризация
    
    Достаточно не сложно представить, что данная методика может абсолютно некорректно работать на более низкиз уровнях, то есть если мы попытаемся кластеризовать пользователей внутри полученных глобальных кластеров, если у нас будет несколько "полюсов" пользования платежами и они будут на больших расстояниях друг от друга, то метка будет метаться между ними где-то в центре, несколько смещаясб в ту сторону, где частота выше, однако это нам вряд ли поможет, поэтому я подумал о следующей модификации, которая опять основывается на времени.
    Первое, что приодит на ум, это то, что достаточно странная затея пытаться описать несколько удаленных точек на карте одной глобальной координатой, она не будет в достаточной степени информативна, поэтому предлагаю отойти в некой степени от того, что мы должны получить всего одну точку, а наоборот попытаемся описать как можно больше мест основываясь на регулярных платежах.
    Мотивация: предполагаем, что у большинства людей одна недля в достаточной степени похожа на другую, то есть работа достаточно регулярна и её время плюс минус фиксировано.
    
    Дефолтное разбиение:
    
        Разбиение по дням недели:
        (понедельник - пятница) - рабочие дни рассматриваем их как одно целое
        (суббота - воскресенье) - выходные дни, рассматриваем их как целое
        
        Разбиение по временным промежуткам:
        (00:00-05:00) - ночь
        (05:00-10:00) - утро
        (10:00-18:30) - день (мотивация - примерное время рабочего дня, неплохо было бы как-то уточнить информацию)
            Замечание:
                1. Можно внутрь добавить перерыв на обед (13:00-15:00)
                2. Определение дня для выходных и рабочих дней может разниться
        (18:30-00:00) - вечер
        
    Зачем всё это нужно?
    
        Как было написано, мы предполагаем, что в платежах присутствуют какие-то регулярные паттерны, которые должны повторяться от недели к неделе, например попить кофе утром по дороге на работу в одном и том же метсе, снять наличные в банкомате рядом с работой, пообедать в одном и том же месте  рабочий перерыв, что может помочь нам определить, в каком месте вообще примерно человек находится в заданные дни недели в заданные временные промежутки.
        
        Также как было отмечено, у нас имеется двойное разбиение на части недели и время дня, соответсвенно мы имеем 8 групп внутри которых мы можем кластеризовать, например, (рабочий деньб, утро) или (выходной день, вечер), для каждой из таких пар мы ставим в соответсвии глобальную координату, при этом можно использовать как предложенный мной метод, так и обычное среднее, однако следует отметить, что в случае  использования моего метода следует предполагать, что всё происходит будто в один день, последовательность обновлений при этом сохраняется, пример:
        
        платёж 1: 26-09-19 07:53:43
        платёж 2: 28-09-19 08:05:31
        
        minutes = 12, дни не учитываются, если обе даты относятся в одной группе дней недели
        
    Как всё это делать?
        
        Преобразовать даты к дням недели, потом делать выборку групп удовлетворяющих разбиениям, рассчитывать внутри полученной выборки восемь глобальных координат, и произвести 8 кластеризаций внутри каждого глобального кластера по всей России.
        
    Проблема:
    
        Действительно серьёзной мне видится проблема людей, которые работают посменно, над тем как это учитыватья ещё не думал

        Касательно людей, работающих по сменам, можно предложить критерии разброса внутри каждого расписания. 
        Предположим, что человек работает k дней через k(посменно), в таком случае мы предполагаем, что в периоды тех к дней, когда он работает, его активность с сервисами банка будет организована одним образом, а когда он находится не на работе пространственная организация его активностей будет локализована в другой области, то есть мы полагаем, что они будут кучковаться, расстояние между точками в рассматриваемых интервалах времени будет меньше, чем если бы мы брали интервалы, в которые он как работает, так и не работает, что в принципе должно неплохо отражать дисперсия координат платежей, то есть идея состоит в том, чтобы просматривать заранее заданные интервалы(списком или специальным алгоритмом) и искать в каких из них наблюдается наименьший разброс в активности, если мы выберем такой интервал, то мы можем с достаточной уверенностью полагать, что будущие платежи клиента будут примерно укладываться в заданные рамки.
    Естественным образом возникает вопрос, а если у клиента изменится график работы то, его новая модель платежей изменится соответственно и мы будем из раза в раз получать сообщения о том, что действие клиента отклоняется от обычного. 
    Тут следует заметить, что решения по типу движущегося окна по датам, по которому определяется глобальная координата или метод экспоненциального сглаживания не исправят ситуацию, потому что мы путём вычисления дисперсии задали интервалы во времени, по которым рассчитывается координата. 
    Отсюда следует вывод, что необходимо отсеживать изменение наиболее оптимального интервала. Можно использовать тот же метод следующего окна, который будет следовать за клиентом во времени и рассчитывать новые дисперсии и на их основе изменять расчёты глобальной координаты, следует отметить,что данная методика очень сильно подвержена влиянию временных изменений в модели активности, если взять окно достаточно коротким, например в пять дней.
        Предположим, что человек работал пн-пт с 9 до 17, в какой-то момент он заболел и не ходил на работу неделю, сидел дома. В таком случае всей его платежи в момент болезни той недели с пн по пт будут выходить за рамки привычного, что мб нежелательно для нашей модели, более того, если выбранное окно по длине меньше либо равно 7 дням и перерасчёт происходит раз в неделю, например по воскресеньям, то на будущей неделе, когда он вернется на привычную работу мы получим для него опять сдвинутые глобальные карты активности.

In [ ]:
import pyreadstat as py

df, meta = py.read_sav('file.SAV')

 В ячейке ниже можно найти функцию:
 
$$
\theta(\log^2_\kappa(1+time_{t}-time_{t-1})-\beta)
$$

Называется new_sigmoid, однако из-за отсутствия зависимости от расстояния, если платежи приходят часто, то сходимость оставляет желать лучшего, если происходит на большое перемещение на большое расстояние за короткий промежуток времени

In [2]:
# Полезная функция расчёта временного расстояния с учётом дней (высокоуровневая)
import math
from datetime import datetime

def minutes_between_high(d1, d2):
    d1 = datetime.strptime(d1, "%H:%M:%S %Y-%m-%d")
    d2 = datetime.strptime(d2, "%H:%M:%S %Y-%m-%d")
    dif = abs(d2-d1)
    #print(dif, d1,d2)
    return (dif.days*24*60+dif.seconds//60)

# Низкоуровневая

def minutes_between_low(d1, d2):
    d1 = datetime.strptime(d1, "%H:%M:%S %Y-%m-%d")
    d2 = datetime.strptime(d2, "%H:%M:%S %Y-%m-%d")
    dif = abs(d2-d1)
    return (dif.seconds//60)

#Формула для расчёта альфа

def sigmoid_dt(x,y):
    #x,y - дата и время в виде строки, формат в функциях выше, разность всегда положительная
    # o - основание логарифма
    # b - beta
    o= 3.34
    b = 5.6
    return 1.0/(1+math.exp(-(math.log(minutes_between_high(y,x)+o,o)-b)))

def new_sigmoid(x,y):
    k=90
    b=4.5
    print(math.log(1+minutes_between_high(y,x),k)**2)
    return 1.0/(1+2**(-(math.log(1+minutes_between_high(y,x),k)**3-b)))


''

     У меня появилось предложение модифицировать дефолтную сигмоиду, путем замены e на другое число, например, на 2, в таком случае, поскольку сигмоида это по факту лишь своеобразное определение, какую часть занимает х по отношение к единице
    
$$
\sigma = \frac{1}{(1+\exp^{-x})} = \frac{\exp^x}{(\exp^x + \exp^0)}
$$

     То почему бы в таком случае не уменьшить основание степенной функции, тогда новая функция сигмоиды не будет так быстро добегать до относительно больших значений в начале. Ниже добавил интерактивный график визуализирующий воздействие на поведение функции основания степенной функции, основания логарифма и байеса.

In [1]:
#!pip install ipywidgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

#2, 4.4, 5.0

def sig_k(x,k):
    return 1/(1+k**(-x))

def f(osn_step, osn_log, bias):
    x = np.arange(0,10080, dtype=float)
    plt.plot(x, sig_k(np.log(x+osn_log)/np.log(osn_log)-bias, osn_step) )

        Ещё одно предложение, которое тут появилось это использовать градиентную оптимизацию для поиска параметров, правда, есть проблема в том, что тк нет протсроанства ответов, то всё довольно печально, сейчас буду думать, как сопоставлять ответы

        Первое, что хочется заменить, что ответом служит альфа во всех случаях, то есть надо придумать правила, по которым выбирается альфа. Оно должно зависеть и от времени и от расстояния. тренировка будет проводиться на данных из исходного датасета с использование pytorch:
        
        Итак, как сформировать альфа
        
        Надо ввести градацию для альфа:
        
        1. Альфа никогда не должна быть равна 1, есть предложение ограничить её 0.95 или 0.9
        
        2. Предлагается взять градацию для времени и расстояния, для каждого из них отдельно ставится альфа, потом они складываются с некими коэффициентами, чтобы не заморачиваться с их подбором возьму 0.35 для времени и 0.65 для расстояния.( данный выбор обосновывается некоторыми наблюдениями полученными в ходе экспериментов)
       
$$
\alpha_{target} = 0.35*\alpha_{time}+0.65*\alpha_{distance}
$$

        Как говорилось ранее попробуем составить градацию, начнем со времени:
        
        1. Где-то в начале предлагалось брать 21 день как то период, который мы считаем большим, поэтому положим альфа_время равным 1, при прошествии больше 21 дня с прошлой активности.
        
        2. 19-21 день альфа равным 0.9
        
        3. Как адекватно дальше распределить сложнос сказать, поэтому зайдем с другой стороны:
        
        - по прошествии одного часа альфа должен составлять 0.025 (alpa_hour)
        
        - при этом при прошествии дня пусть альфа будет равен 0.2 (alpha_day)
        
        - основываясь на этом для других промежутков времени между ними альфа будет определяться как
$$        
\alpha = alpha_{hour} + (t-60)*\frac{(alpha_{day}-alph_{hour})}{23*60}
$$

        - если предлагаемые значения альфа день и час не устраивают, можно смело менять
        
        - следующий рубеж 3 дня, пусть он будет 0.35 (alpha_3d)
        
$$        
\alpha = alpha_{day} + (t-24*60)*\frac{(alpha_{3d}-alph_{day})}{2*24*60}
$$

        - следующий рубеж - неделя, 0.5 (alpha_week)
        
$$        
\alpha = alpha_{3d} + (t-3*24*60)*\frac{(alpha_{week}-alph_{3d})}{4*24*60}
$$      

        - две недели, 0.75 (alpha_2w)

$$        
\alpha = alpha_{week} + (t-7*24*60)*\frac{(alpha_{2w}-alph_{week})}{7*24*60}
$$    

        - 18 дней, 0.9 (alpha_18d) как было написано раньше
        
$$        
\alpha = alpha_{2w} + (t-14*24*60)*\frac{(alpha_{18d}-alph_{2w})}{4*24*60}
$$    

        - 21 день, 0.95 (alpha_end)
        
$$        
\alpha = alpha_{18d} + (t-18*24*60)*\frac{(alpha_{end}-alph_{18d})}{3*24*60}
$$            
        - ну и тогда если больше 21 дня 
$$
\alpha = 1
$$

        Напоминаю, что не стоит волновать, что альфа равен для времени единице, так как значение временного альфа не может превысить 0.35

        Теперь проделаем такую же градацию для расстояния:
        
        1. Пусть альфа равно 1 когда расстояние от нового платежа до глобальной координаты >2500 км
        
             - 0.9 когда расстояние больше 2000
             
             - 0.85 - 1500 км
             
             - 0.8 - 1000 км
             
             - 0.75 - 500 км
             
             - 0.65 - 300 км
             
             - 0.5  - 100 км
             
             - 0.4  - 75  км
             
             - 0.35 - 50  км
             
             - (0, 50)км - вычисляется линейно, как и все промежутки в описанной градации
             
$$
\alpha_{distance} = alpha_{old} + \frac{alpha_{new}-alpha_{old}}{porog_{new}-porog_{old}}*(platezh - global)
$$

$$        
\alpha = alpha_{hour} + (t-60)*\frac{(alpha_{day}-alph_{hour})}{23*60}
$$

In [3]:
def target_alpha_func(lat, lon, time, glat, glon, ptime):
    minutes = minutes_between_high(time, ptime)
    km = dist_km(lat, lon, glat, glon)
    
    days = minutes//(24*60)
    
    a_time = 0.0
    a_dist = 0.0
    
    a_hour = 0.025
    a_day  = 0.2
    a_3d   = 0.35
    a_week = 0.5
    a_2w   = 0.75
    a_18d  = 0.9
    a_end  = 0.95
    
    a_50 = 0.35
    a_75 = 0.4
    a_100 = 0.5
    a_300 = 0.65
    a_500 = 0.75
    a_1000 = 0.8
    a_1500 = 0.85
    a_2000 = 0.9
    a_2500 = 0.95
    
    kek = 0.3
    lol = 0.7
    
    
    if(days<1):
        hours = minutes//60
        if(hours<1):
            a_time = minutes*(a_hour/60.0)
        else:
            a_time = a_hour + (minutes-60)*(a_day-a_hour)/(23*60.0)
    elif(days<3):
        a_time = a_day + (minutes - 24*60)*(a_3d-a_day)/(2*24*60.0)
    elif(days<7):
        a_time = a_3d  + (minutes - 3*24*60)*(a_week-a_3d)/(4*24*60.0)
    elif(days<14):
        a_time = a_week + (minutes - 7*24*60)*(a_2w-a_week)/(7*24*60.0)
    elif(days<18):
        a_time = a_2w +(minutes - 14*24*60)*(a_18d-a_2w)/(4*24*60)
    elif(days<21):
        a_time = a_18d+(minutes - 18*24*60)*(a_end-a_18d)/(3*24*60)
    else:
        a_time=1.0
        
    if(km<=50):
        a_dist = km*a_50/50.0
    elif(km<=75):
        a_dist = a_50+(km-50)*(a_75-a_50)/(75-50)
    elif(km<=100):
        a_dist = a_75+(km-75)*(a_100-a_75)/(100-75)
    elif(km<=300):
        a_dist = a_100+(km-100)*(a_300-a_100)/(300-100)
    elif(km<=500):
        a_dist = a_300+(km-300)*(a_500-a_300)/(500-300)
    elif(km<=1000):
        a_dist = a_500+(km-500)*(a_1000-a_500)/(1000-500)
    elif(km<=1500):
        a_dist = a_1000+(km-1000)*(a_1500-a_1000)/(1500-1000)
    elif(km<=2000):
        a_dist = a_1500+(km-1500)*(a_2000-a_1500)/(2000-1500)
    elif(km<=2500):
        a_dist = a_2000+(km-2000)*(a_2500-a_2000)/(2500-2000)
    else:
        a_dist = 1.0
        
    return kek*a_time+lol*a_dist

In [4]:
#!pip install torch

In [5]:
import torch
import torch.optim as optim
import math

#ОБЪЯВЛЕНИЕ ГИПЕРПАРАМЕТРОВ и переделывание функций под pytorch

# Для первой альфа

k = torch.nn.Parameter(torch.ones(1)*3.3,requires_grad=True)
beta = torch.nn.Parameter(torch.ones(1)*5.6, requires_grad=True)
step_osn = torch.nn.Parameter(torch.ones(1)*math.e, requires_grad=True)

#Для второй альфа (вариант 1)
zeta_1 = torch.nn.Parameter(torch.ones(1)*0.3,requires_grad=True)
eta_1 = torch.nn.Parameter(torch.ones(1)*0.7,requires_grad=True)
k_1 = torch.nn.Parameter(torch.ones(1)*3.3,requires_grad=True)
beta_1 = torch.nn.Parameter(torch.ones(1)*5.6, requires_grad=True)
step_osn_1 = torch.nn.Parameter(torch.ones(1)*math.e, requires_grad=True)

#Для третьей альфа (вариант 2)

zeta_2 = torch.nn.Parameter(torch.ones(1)*0.3,requires_grad=True)
eta_2 = torch.nn.Parameter(torch.ones(1)*0.7,requires_grad=True)
beta_1_2 = torch.nn.Parameter(torch.ones(1)*5.6,requires_grad=True)
beta_2_2 = torch.nn.Parameter(torch.ones(1)*4,requires_grad=True)
k_2 = torch.nn.Parameter(torch.ones(1)*3.5,requires_grad=True)
step_osn_1_2 = torch.nn.Parameter(torch.ones(1)*math.e,requires_grad=True)
step_osn_2_2 = torch.nn.Parameter(torch.ones(1)*math.e,requires_grad=True)

#__________________________________________________________________________________________

#Функции для pytorch


def sigmoid_dt_(x,y, k = torch.ones(1)*3.3, beta = torch.ones(1)*5.6, step_osn = torch.ones(1)*math.e):
    #x,y - дата и время в виде строки, формат в функциях выше, разность всегда положительная
    # k - основание логарифма
    # b - beta
    dist = minutes_between_high(y,x)
    if(k.data<=1):
        k = torch.nn.Parameter(torch.ones(1, requires_grad=True))*1.1
    return 1.0/(1+step_osn.pow(-(torch.log(dist+k)/torch.log(k)-beta)))


def sigmoid_dt_var_2_(lat_1, lon_1, dtime_1, lat_2, lon_2, dtime_2,
                     zeta = torch.ones(1)*0.3, 
                     eta = torch.ones(1)*0.7,
                     beta_1 = torch.ones(1)*5.6,
                     beta_2 = torch.ones(1)*4,
                     k = torch.ones(1)*3.5,
                     step_osn_1 = torch.ones(1)*math.e,
                     step_osn_2 = torch.ones(1)*math.e
                    ):#вариант 1
    
    #Коэффициенты при сигмоидах
    #zeta = zeta
    #eta = eta
    
    
    #Свободный член
    #beta_1 = beta_1
    #beta_2 = beta_2
    
    #Основание логарифма
    #k = k
    
    km = dist_km(lat_1, lon_1, lat_2, lon_2)
    dist = minutes_between_high(dtime_1, dtime_2)
    
    sigmoid_1 = 1.0/(1.0+step_osn_1.pow(-(torch.log(dist+k)/torch.log(k)-beta_1)))
    sigmoid_2 = 1.0/(1.0+step_osn_2.pow(-(torch.log1p(torch.ones(1)*km)/torch.log(torch.ones(1)*10.0)-beta_2)))
    
    
    return zeta*sigmoid_1+eta*sigmoid_2
    
def sigmoid_dt_var_1_(lat_1, lon_1, dtime_1, lat_2, lon_2, dtime_2,
                     zeta = torch.ones(1)*0.3, 
                     eta = torch.ones(1)*0.7, 
                     beta =torch.ones(1)*5, 
                     k = torch.ones(1)*3.5, 
                     step_osn = torch.ones(1)*math.e):# вариант 2
    
    #Коэффициенты при логарифмах
    #zeta = zeta
    #eta = eta
    # так как эти коэффициенты внутри сигмоиды, их сумма не обязательно должна быть равна 1
    
    #Свободный член
    #beta = beta
    
    #Основание логарифма
    #k = k
    
    
    
    km = dist_km(lat_1, lon_1, lat_2, lon_2)
    dist = minutes_between_high(dtime_1,dtime_2)
    
    if(k.data<=1):
        k = torch.nn.Parameter(torch.ones(1, requires_grad=True))*1.1
    
    log_t = torch.log(dist+k)/torch.log(k)
    log_d = torch.log1p(torch.ones(1)*km)/torch.log(torch.ones(1)*10.0)
    
    
    #print(log_d, 'логарифм расстояния')
    
    return 1.0/(1.0+step_osn.pow(-(zeta*log_t+eta*log_d-beta)))



In [6]:
from random import shuffle
import torch
import torch.optim as optim
import pandas as pd
from IPython.display import clear_output


#Optimizers

o_a  = optim.Adam([k, beta, step_osn],lr=1e-2)
o_a1 = optim.Adam([zeta_1, eta_1, k_1, beta_1, step_osn_1],lr=1e-2)
o_a2 = optim.Adam([zeta_2, eta_2, k_2, beta_1_2, beta_2_2, step_osn_1_2, step_osn_2_2],lr=1e-2)



#Reading data and gatting list of unique ids, shuffle them
data = pd.read_csv("/home/lars/Data.csv").dropna()
names = list(set(data['user_id']))
shuffle(names)




def parameters():
    
    o_a.zero_grad()
    o_a1.zero_grad()
    o_a2.zero_grad()
    shift = len(names)//100000
    
    for i in range(100000):
        srez = names[i*shift:(i+1)*shift]#getting next set of ids
        
        for name in srez:#going through batch
            
            buf = data.loc[data['user_id']==name]
            buf = buf.groupby(['user_id','Date', 'Time']).apply(pd.DataFrame.reset_index)
            buf = buf.iloc[:,[1,2,3,4,5]].reset_index(drop=True)
            
            global_lat = 0.0
            global_lon = 0.0
            
            tglobal_lat = 0.0
            tglobal_lon = 0.0
            
            global_lat_var_1 = 0.0
            global_lon_var_1 = 0.0
            
            global_lat_var_2 = 0.0
            global_lon_var_2 = 0.0
            
            for j in range(buf.shape[0]):
                lat = buf.iloc[j,1]
                lon = buf.iloc[j,2]
                t_d = buf.iloc[j,3]+" "+buf.iloc[j,4]
                if(j == 0):
                    global_lat = lat
                    global_lon = lon
                    p_td = t_d
                else:
                    alpha = sigmoid_dt_(p_td,t_d, k=k, beta = beta, step_osn=step_osn)
                    
                    alpha_var_1 = sigmoid_dt_var_1_(lat, lon, t_d, global_lat, global_lon, p_td, k = k_1,
                                                beta=beta_1, zeta = zeta_1, eta=eta_1, step_osn = step_osn_1)
                    
                    alpha_var_2 = sigmoid_dt_var_2_(lat, lon, t_d, global_lat, global_lon, p_td,
                                                k = k_2, beta_1=beta_1_2, beta_2=beta_2_2,
                                                zeta = zeta_2,
                                                eta=torch.clamp(1-zeta_2,min=0.0, max=1.0),
                                                step_osn_1 = step_osn_1_2,
                                                step_osn_2 = step_osn_2_2)
                    
                    target_alpha = target_alpha_func(lat,lon,t_d,global_lat, global_lon, p_td)
                        #alpha_2 = sigmoid_dt_2(p_td,t_d)
                        #new_sigmoid(p_td,t_d)#
                        
                    tglobal_lat = (target_alpha*lat+(1.0-target_alpha)*tglobal_lat)
                    tglobal_lon = (target_alpha*lon+(1.0-target_alpha)*tglobal_lon)
                    
                    global_lat = (alpha*lat+(1.0-alpha)*global_lat).data
                    global_lon = (alpha*lon+(1.0-alpha)*global_lon).data
                    
                    
                    global_lat_var_1 = (alpha_var_1*lat+(1.0-alpha_var_1)*global_lat_var_1).data
                    global_lon_var_1 = (alpha_var_1*lon+(1.0-alpha_var_1)*global_lon_var_1).data

                        #print('alpha_var_2:',alpha_var_2)
                    global_lat_var_2 = (alpha_var_2*lat+(1.0-alpha_var_2)*global_lat_var_2).data
                    global_lon_var_2 = (alpha_var_2*lon+(1.0-alpha_var_2)*global_lon_var_2).data
                    
                    loss_a   = ((target_alpha-alpha)*100).pow(2)*1.0/shift
                    loss_a_1 = ((target_alpha-alpha_var_1)*100).pow(2)*1.0/shift
                    loss_a_2 = ((target_alpha-alpha_var_2)*100).pow(2)*1.0/shift
                    
                    loss_a.backward()
                    loss_a_1.backward()
                    loss_a_2.backward()
        
        o_a.step()
        o_a1.step()
        o_a2.step()
        
        o_a.zero_grad()
        o_a1.zero_grad()
        o_a2.zero_grad()
        
        clear_output()
        print("Iteration: "+str(i))
        print("Default alpha: ")
        print("Основание логарифма: "+str(k.data))
        print("Основание степени:   "+str(step_osn.data))
        print("beta:                "+str(beta.data))
        print(" ")
        print("Вариант 1")
        print("Основание логарифма: "+str(k_1.data))
        print("Основание степени:   "+str(step_osn_1.data))
        print("beta:                "+str(beta_1.data))
        print("zeta:                "+str(zeta_1.data))
        print("eta:                 "+str(eta_1.data))
        print(" ")
        print("Вариант 2")
        print("Основание логарифма:   "+str(k_2.data))
        print("Основание степени 1:   "+str(step_osn_1_2.data))
        print("Основание степени 2:   "+str(step_osn_2_2.data))
        print("beta_1:                "+str(beta_1_2.data))
        print("beta_2:                "+str(beta_2_2.data))
        print("zeta:                  "+str(zeta_2.data))
        print("eta:                   "+str((1-zeta_2).data))

In [7]:
#обучение остановлено после 4 часов в связи с началом осциляции вокруг определённых значений
if __name__=='__main__':
    parameters()

Iteration: 191
Default alpha: 
Основание логарифма: tensor([4.5584])
Основание степени:   tensor([2.0762])
beta:                tensor([7.0910])
 
Вариант 1
Основание логарифма: tensor([3.1446])
Основание степени:   tensor([2.7475])
beta:                tensor([5.5062])
zeta:                tensor([0.4776])
eta:                 tensor([0.7551])
 
Вариант 2
Основание логарифма:   tensor([4.1485])
Основание степени 1:   tensor([3.9043])
Основание степени 2:   tensor([3.0817])
beta_1:                tensor([6.5624])
beta_2:                tensor([2.5812])
zeta:                  tensor([0.3083])
eta:                   tensor([0.6917])


KeyboardInterrupt: 

In [10]:
#Значения пока тестовые, я остановил алгоритм, чтобы их глянуть

print("Default alpha: ")
print("Основание логарифма: "+str(k.data))
print("Основание степени:   "+str(step_osn.data))
print("beta:                "+str(beta.data))
print(" ")
print("Вариант 1")
print("Основание логарифма: "+str(k_1.data))
print("Основание степени:   "+str(step_osn_1.data))
print("beta:                "+str(beta_1.data))
print("zeta:                "+str(zeta_1.data))
print("eta:                 "+str(eta_1.data))
print(" ")
print("Вариант 2")
print("Основание логарифма:   "+str(k_2.data))
print("Основание степени 1:   "+str(step_osn_1_2.data))
print("Основание степени 2:   "+str(step_osn_2_2.data))
print("beta_1:                "+str(beta_1_2.data))
print("beta_2:                "+str(beta_2_2.data))
print("zeta:                  "+str(zeta_2.data))
print("eta:                   "+str((1-zeta_2).data))

Default alpha: 
Основание логарифма: tensor([4.5584])
Основание степени:   tensor([2.0762])
beta:                tensor([7.0910])
 
Вариант 1
Основание логарифма: tensor([3.1446])
Основание степени:   tensor([2.7475])
beta:                tensor([5.5062])
zeta:                tensor([0.4776])
eta:                 tensor([0.7551])
 
Вариант 2
Основание логарифма:   tensor([4.1485])
Основание степени 1:   tensor([3.9043])
Основание степени 2:   tensor([3.0817])
beta_1:                tensor([6.5624])
beta_2:                tensor([2.5812])
zeta:                  tensor([0.3083])
eta:                   tensor([0.6917])
